# SHFQA

Just like the driver for the HDAWG in the previous example, we now use the `zhinst.qcodes.SHFQA` instrument driver.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
import zhinst.qcodes as ziqc

shfqa = ziqc.SHFQA("shfqa", "dev12036", interface="1gbe", host="localhost")

Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV12036 on interface 1GBE
Connected to: Zurich Instruments SHFQA (serial:dev12036, firmware:67774) in 8.45s


In [2]:
print([k for k in shfqa.submodules.keys()])
print([k for k in shfqa.parameters.keys()])

['qachannels', 'scope', 'stats', 'status', 'system', 'features', 'dios']
['IDN', 'data_server_version', 'firmware_version', 'fpga_version', 'sw_trigger', 'ref_clock', 'ref_clock_actual', 'ref_clock_status', 'clockbase']


## Channel parameters of the SHFQA

SHFQA has 2 or 4 channels, each of them can serve for a single readout line. Each QAchannel module can readout up to 8 or 16 quits.

In [3]:
print([k for k in shfqa.qachannels[0].parameters.keys()])

['input', 'input_range', 'output', 'output_range', 'center_freq', 'mode']


In [4]:
shfqa.qachannels[0].input_range(0)
shfqa.qachannels[0].output_range(-5)
shfqa.qachannels[0].center_freq(5e9)
shfqa.qachannels[0].input('on')
shfqa.qachannels[0].output('on')
# test what are correctly printed
print(shfqa.qachannels[0].input_range.__doc__)

Node: /DEV12036/QACHANNELS/0/INPUT/RANGE
Description: Sets the maximal Range of the Signal Input power. The instrument selects the closest available Range with a resolution of 5 dBm.
Type: Double
Properties: Read, Write, Setting
Unit: dBm


Parameter class:

* `name` input_range
* `label` Maximal Range of the Signal Input Power
* `unit` dBm
* `vals` None


## Spectroscopy mode of the SHFQA
SHFQA has two application modes, **Spectroscopy** and **Readout**. Spectroscopy mode is generally used for resonator spectroscopy experiments, and Readout mode is used for qubit readout experiments with fixed readout frequencies. 

In Spectroscopy mode, offset frequency sweep is done by the **sweeper** module. Each frequency sweep is triggered by an internal or external trigger. External trigger is recommended. With **shfqa.set_trigger_loopback()** a marker output is connected to a trigger input internally without any physical connections. 

In [5]:
print([k for k in shfqa.qachannels[0].sweeper.parameters.keys()])

['oscillator_gain', 'oscillator_freq', 'integration_time', 'integration_length', 'integration_delay', 'trigger_source']


In [6]:
shfqa.qachannels[0].mode('spectroscopy')
sweeper0=shfqa.qachannels[0].sweeper

shfqa.set_trigger_loopback()
sweeper0.trigger_source("channel0_trigger_input0")
sweeper0.trigger_level(0)
sweeper0.trigger_imp50(1)

sweeper0.start_frequency(-200e-6)
sweeper0.stop_frequency(200e6)
sweeper0.num_points(51)
sweeper0.mapping("linear")
sweeper0.integration_time(100e-6)
sweeper0.num_averages(2)
sweeper0.averaging_mode("sequential")
# print(sweeper0.mapping.__doc__)

In [7]:
sweeper0.run()
result=sweeper0.read()
# sweeper0.plot()

Run a sweep with 51 frequency points in the range of [-2e-10, 200.0] MHz + 5.0 GHz. 
Mapping is linear. 
Integration time = 0.0001 sec. 
Measures 2 times per frequency point. 
Averaging mode is sequential.


## Readout mode of the SHFQA
In Readout mode, **generator** module is used to configure waveform playback, such as upload waveforms, construct and compile readout sequences. Integration parameters and result source are configured by **readout**. Please note that the data type of uploaded waveforms has to be **complex128**.

In [8]:
shfqa.qachannels[0].mode('readout')
generator0 = shfqa.qachannels[0].generator
readout0 = shfqa.qachannels[0].readout

In [9]:
print([k for k in shfqa.qachannels[0].generator.parameters.keys()])
print([k for k in shfqa.qachannels[0].readout.parameters.keys()])
# help(generator0)
# help(readout0)

['dig_trigger1_source', 'dig_trigger2_source', 'playback_delay', 'single']
['integration_length', 'integration_delay', 'result_source']


In [10]:
num_readouts = 100
pulse_duration = 100e-9
readout_freq = 200e6
sampling_rate = 2e9

pulse = 0.5*np.exp(2j*np.pi*readout_freq*np.linspace(0, pulse_duration, int(pulse_duration * sampling_rate)))
weight = np.conj(pulse)

shfqa.set_trigger_loopback()
generator0.dig_trigger1_source("chan0trigin0")
# Delay between receving the trigger and playing the readout pulses
generator0.playback_delay(0) 

#Define the program
seqc_program ="""
repeat($param1$) {
    waitDigTrigger(1);
    startQA(QA_GEN_ALL, QA_INT_ALL, true, 0, 0x0);
}    
"""
generator0.set_sequence_params(
    sequence_type="Custom",
    program = seqc_program ,
    custom_params = [num_readouts],
)

# Upload readout pulse and integration weight
generator0.reset_queue()
generator0.queue_waveform(pulse)
generator0.compile()
generator0.upload_waveforms()
readout0.integrations[0].set_int_weights(weight)

Current length of queue: 1
Upload of 1 waveforms took 0.18024 s


In [11]:
setup_delay = 200e-9

readout0.integration_length(len(weight))
readout0.integration_delay(setup_delay)
readout0.result_source("result_of_integration")

readout0.arm(length = num_readouts, averages = 1)
generator0.stop()
generator0.run()
    
result = readout0.read()

## Scope monitor of the SHFQA

The **scope** module of the SHFQA is used to monitor or record time traces of signals at IF frequency down converted by the SHFQA. 

In [12]:
print([k for k in shfqa.scope.parameters.keys()])
# help(shfqa.scope)

['channel1', 'channel2', 'channel3', 'channel4', 'input_select1', 'input_select2', 'input_select3', 'input_select4', 'trigger_source', 'trigger_delay', 'length', 'time']


In [13]:
scope=shfqa.scope
scope.channel1('on')
scope.input_select1
scope.trigger_source("channel0_trigger_input0")
scope.trigger_delay(200e-9)
scope.length(1024)
scope.averaging(1)
scope.segments(1)
scope.run()
result=scope.read()
# scope.stop()

## DIO of the SHFQA
The **dios** of the SHFQA is used to communicate qubit readout results from SHFQA to qubit control instruments, such as SHFSG or HDAWG.

In [14]:
print([k for k in shfqa.dios[0].parameters.keys()])

['drive', 'output', 'mode', 'input', 'interface']
